# How to Render a Metal Design into Ansys

## 1. Perform the necessary imports and create a design in Metal first.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [ ]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

QAnsysRenderer.default_options

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket

In [ ]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs.

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.components.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2a. Render into Ansys HFSS.
### Registration occurs automatically. Must have an open Ansys project and HFSS eigenmode design before executing render_design. To create a new Ansys project, click "New", the leftmost icon right below the file menu. To create a new HFSS design, select "Insert HFSS Design" under Project in the file menu. Right click on the Ansys design in the left panel, go to "Solution Type...", and choose Eigenmode.

In [ ]:
fourq_hfss = design.renderers.hfss # Access Ansys HFSS renderer

In [ ]:
fourq_hfss.open_ansys_design() # Open either a new or existing design based on default options

### Note: Run only one of the three cells below at a time. Clear everything from the Ansys design before rendering a new selection by clicking in the Ansys GUI -> holding down Ctrl + A (to select everything in it) -> right clicking -> Edit -> Delete. Check that the correct Ansys design is active by double clicking on it in the leftmost panel. If switching between designs, re-run the 2 cells above before starting to render.

In [ ]:
fourq_hfss.render_design([], []) # Render the whole Metal design into Ansys HFSS.

In [ ]:
fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')]) # Render a single qubit with 2 endcaps into Ansys HFSS.

In [ ]:
fourq_hfss.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')]) # Render 2 qubits and 2 endcaps, one per qubit, into Ansys HFSS.

In [ ]:
fourq_hfss.render_design(['Q2'], [('Q2', 'c')], [('Q2', 'b', 70)])

### Render a single qubit with an added port and open endcap into Ansys HFSS for a Driven Modal solution type. But first, right click on the Ansys design in the left panel, go to "Solution Type...", and choose Modal.

In [ ]:
fourq_hfss.render_design(['Q2'], [('Q2', 'c')], [('Q2', 'b', 70)]) # Set the lumped port impedance to 70 Ohms.

## 2b. Render into Ansys Q3D.
### Registration occurs automatically. Must have an open Ansys project and Q3D design before executing render_design. To create a new Ansys project, click "New", the leftmost icon right below the file menu. To create a new Q3D design, select "Insert Q3D Extractor Design" under Project in the file menu.

In [ ]:
fourq_q3d = design.renderers.q3d # Access Ansys Q3D renderer

In [ ]:
fourq_q3d.open_ansys_design() # Open either a new or existing design based on default options

### Note: Run only one of the three cells below at a time. Clear everything from the Ansys design before rendering a new selection by clicking in the Ansys GUI -> holding down Ctrl + A (to select everything in it) -> right clicking -> Edit -> Delete. Check that the correct Ansys design is active by double clicking on it in the leftmost panel. If switching between designs, re-run the 2 cells above before starting to render.

In [ ]:
fourq_q3d.render_design([], []) # Render the whole Metal design into Ansys Q3D.

In [ ]:
fourq_q3d.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')]) # Render a single qubit with 2 endcaps into Ansys Q3D.

In [ ]:
fourq_q3d.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')]) # Render 2 qubits and 2 endcaps, one per qubit, into Ansys Q3D.

## 3b. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix. 
### This follows from step 2b above.

In [ ]:
fourq_q3d.add_q3d_setup() # Add a solution setup.

In [ ]:
fourq_q3d.analyze_setup("Setup") # Analyze said solution setup.

In [ ]:
fourq_q3d.get_capacitance_matrix() # Get its capacitance matrix.

In [ ]:
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(12.31, 2, 2, 7.0, [0.1, 0.1, 0.1], 9)
dict_lom